In [1]:
# !mkdir data
# !wget http://www.manythings.org/anki/rus-eng.zip -O 'data/rus-eng.zip'
# !wget https://object.pouta.csc.fi/OPUS-OpenSubtitles/v1/moses/en-ru.txt.zip -O 'data/en-ru.txt.zip'

In [2]:
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd
import zipfile
from collections import Counter
import torch
from torch.nn.utils.rnn import pad_sequence
import re

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# dataset_anki_path = 'data/rus-eng.zip'
# with zipfile.ZipFile(dataset_anki_path, 'r') as zip_ref:
#     zip_ref.extractall('anki_data')
# dataset_opensub_path = 'data/en-ru.txt.zip'
# with zipfile.ZipFile(dataset_opensub_path, 'r') as zip_ref:
#     zip_ref.extractall('opensubtitles_data')

In [4]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [5]:
def unicodeToAscii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) \
        if unicodedata.category(c) != 'Mn')

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Zа-яёъА-ЯЁЪ.!?]+", r" ", s)
    return s


In [6]:
def load_opensubtitles_data(file_path_ru, file_path_en, num_samples=100_000):
    input_texts = []
    with open(file_path_en, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[:num_samples]:
            en_text = line.strip()
            input_texts.append(normalizeString(en_text))
    target_texts = []
    with open(file_path_ru, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[:num_samples]:
            ru_text = line.strip()
            target_texts.append(normalizeString(ru_text))
    return input_texts, target_texts

def load_anki_data(file_path, num_samples=100_000):
    input_texts = []
    target_texts = []
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[:num_samples]:
            en_text, ru_text = line.strip().split('\t')[:-1]
            input_texts.append(normalizeString(en_text))
            target_texts.append(normalizeString(ru_text)) 
    return input_texts, target_texts

In [7]:
def load_opensubtitles_data(file_path_ru, file_path_en, num_samples=100_000):
    input_texts = []
    with open(file_path_en, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[:num_samples]:
            en_text = line.strip()
            input_texts.append(normalizeString(en_text))
    target_texts = []
    with open(file_path_ru, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[:num_samples]:
            ru_text = line.strip()
            target_texts.append(normalizeString(ru_text))
    return input_texts, target_texts

def load_anki_data(file_path, num_samples=100_000):
    input_texts = []
    target_texts = []
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[:num_samples]:
            en_text, ru_text = line.strip().split('\t')[:-1]
            input_texts.append(normalizeString(en_text))
            target_texts.append(normalizeString(ru_text)) 
    return input_texts, target_texts

In [8]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    anki_input_texts, anki_target_texts = load_anki_data('anki_data/rus.txt')
    opensub_input_texts, opensub_target_texts = load_opensubtitles_data('opensubtitles_data/OpenSubtitles.en-ru.ru',
                                                                 'opensubtitles_data/OpenSubtitles.en-ru.en')
    pairs = list(zip(anki_input_texts, anki_target_texts)) + list(zip(opensub_input_texts, opensub_target_texts))
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [9]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p, n=MAX_LENGTH ):
    return len(p[0].split(' ')) < n and \
        len(p[1].split(' ')) < n and \
        p[0].startswith(eng_prefixes)


def filterPairs(pairs, n=MAX_LENGTH):
    return [pair for pair in pairs if filterPair(pair,n)]

In [10]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)[:10000]
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'rus', False)
print(random.choice(pairs))

Reading lines...
Read 123924 sentence pairs
Trimmed to 10000 sentence pairs
Counting words...
Counted words:
eng 2035
rus 4361
('you re too humble .', 'ты слишком скромная .')


In [11]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
  
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
        

class AttenDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttenDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.wh = nn.Linear(self.hidden_size,self.hidden_size,bias=False)
        self.we = nn.Linear(self.hidden_size,self.hidden_size,bias=False)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size,bias=False)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        self.weight = nn.Parameter(torch.FloatTensor(1,1, self.hidden_size))
        

    def forward(self, input, hidden, encoder_outputs):
        # [1,1] [1,1,512] [10,512]
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        
        attn  = torch.tanh(self.wh(hidden)+self.we(encoder_outputs.unsqueeze(0)))
        scores = torch.bmm(self.weight,attn.transpose(1,2))
        weights = F.log_softmax(scores,2)
        
        context = torch.bmm(weights,encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded, context), 2)
        output = self.attn_combine(output)
                           
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
        
        

In [12]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)
    

In [13]:
def train(input_tensor, target_tensor, encoder, decoder, 
          encoder_optimizer, decoder_optimizer, 
          criterion, max_length=MAX_LENGTH,teacher_forcing_ratio = 0.5):
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            _, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach() 
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break
    loss.backward()
    kc =5 + 100*(1-teacher_forcing_ratio)
    torch.nn.utils.clip_grad_norm_(encoder.parameters(), kc)
    torch.nn.utils.clip_grad_norm_(decoder.parameters(), kc)
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length
            
    

In [14]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [15]:
def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=5e-5):
    start = time.time()
    print_loss_total = 0
    
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()
    for iter_ in range(1, n_iters + 1):
        training_pair = training_pairs[iter_ - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, 
                     criterion,teacher_forcing_ratio = 1-iter_/n_iters)
        print_loss_total += loss
        if iter_ % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter_ / n_iters),
                                         iter_, iter_ / n_iters * 100, print_loss_avg))
            if print_loss_avg < 0.01:
                continue
        
        

In [16]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(
                input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]
                            
        

def evaluateRandomly(encoder, decoder, l, n=3):
    pairs_ = [pair for pair in pairs if len(pair[0].split()) == l]
    for i in range(n):
        
        
        pair = random.choice(pairs_)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

        
        

In [17]:
# [pair for pair in pairs if len(pair[0].split()) == 5]

In [18]:
hidden_size = 256
encoder1 = Encoder(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttenDecoder(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

In [19]:
trainIters(encoder1, attn_decoder1, 100_000, print_every=10000)

8m 10s (- 73m 38s) (10000 10%) 4.3371
22m 29s (- 89m 57s) (20000 20%) 3.9063
274m 46s (- 641m 7s) (30000 30%) 3.6611
288m 4s (- 432m 7s) (40000 40%) 3.4898
302m 23s (- 302m 23s) (50000 50%) 3.3222
315m 55s (- 210m 37s) (60000 60%) 3.2385
331m 35s (- 142m 6s) (70000 70%) 3.1433
340m 19s (- 85m 4s) (80000 80%) 3.0372
347m 5s (- 38m 33s) (90000 90%) 2.9325
353m 49s (- 0m 0s) (100000 100%) 2.8550


In [20]:
# 3 words
evaluateRandomly(encoder1, attn_decoder1, 4)

> she is dead .
= она умерла .
< она она она она она она она она она она

> we re drinking .
= мы пьем .
< <EOS>

> you re mean .
= вы злая .
< . . . . . . . . . .



In [21]:
# 6 words
evaluateRandomly(encoder1, attn_decoder1,7)

> i m so proud of you .
= я так тобои горжусь .
< я я я я я я я я я я

> i m blind in one eye .
= у меня один глаз не видит .
< . . <EOS>

> i m not good at it .
= я в этом не сильна .
< . . . . . . . . . .



In [25]:
# 7 words
evaluateRandomly(encoder1, attn_decoder1,8)

> i m at my wit s end .
= ума не приложу что делать .
< я я я я я я я я я я

> i m at my wit s end .
= не знаю что еще можно предпринять .
< я я я я я я я я я я

> i m as old as he is .
= мы с ним одного возраста .
< мы мне мне мне <EOS>



In [30]:
def get_word_vector(word, encoder, decoder, vocab, device='cpu'):
    input_tensor = torch.tensor([[vocab[word]]], device=device)
    
    encoder_hidden = encoder.initHidden()
    encoder_output, encoder_hidden = encoder(input_tensor[0], encoder_hidden)
    
    return encoder_hidden[0].squeeze(0)

def evaluate_word_pairs(encoder, decoder, vocab, synonym_pairs, antonym_pairs, device='cpu'):
    results = []

    def cosine_similarity_torch(vec1, vec2):
        # Ensure the vectors are 2D for cosine similarity
        if vec1.dim() == 1:
            vec1 = vec1.unsqueeze(0)
        if vec2.dim() == 1:
            vec2 = vec2.unsqueeze(0)
        
        cos = F.cosine_similarity(vec1, vec2, dim=-1)  # Compare along the last dimension
        return cos.item()

    for word1, word2 in synonym_pairs:
        vec1 = get_word_vector(word1, encoder, decoder, vocab, device)
        vec2 = get_word_vector(word2, encoder, decoder, vocab, device)
        cosine_sim = cosine_similarity_torch(vec1, vec2)
        results.append([word1, word2, cosine_sim, "Синонимы"])

    for word1, word2 in antonym_pairs:
        vec1 = get_word_vector(word1, encoder, decoder, vocab, device)
        vec2 = get_word_vector(word2, encoder, decoder, vocab, device)
        cosine_sim = cosine_similarity_torch(vec1, vec2)
        results.append([word1, word2, cosine_sim, "Антонимы"])

    df = pd.DataFrame(results, columns=["Слово 1", "Слово 2", "Косинусное расстояние", "Тип"])
    return df


# Example word pairs
synonym_pairs = [("страшно", "ужасно")]
antonym_pairs = [("страшно", "уверен")]

# Assuming encoder1, attn_decoder1, and output_lang.word2index are defined
df_results = evaluate_word_pairs(encoder1, attn_decoder1, output_lang.word2index, synonym_pairs, antonym_pairs)

print(df_results)


   Слово 1 Слово 2  Косинусное расстояние       Тип
0  страшно  ужасно               0.035930  Синонимы
1  страшно  уверен              -0.010225  Антонимы
